In [ ]:
import os,re
import numpy as np
import torch
import sympy
from fractions import Fraction

#some classes to hold the data in different formats.
#symb is an overload of dict with some elementwise operators on values,
#sumlist is an overload of list with elementwise sum and multiplication operations

class Symb(dict):
    def valmult(self,m,d1):
        return {k:(m*v) for k,v in d1.items()}
    
    def dictmerge(self,d1,d2):
        def getval(k):
            if k in d1 and k in d2: val= d1[k]+d2[k]
            elif k in d1: val= d1[k]
            elif k in d2: val= d2[k]
            else: val = None
            return val

        return{k:getval(k) for k in d1|d2 if getval(k) != 0}
    
    def dictdiff(self,d1,d2):
        def getval(k):
            if k in d1 and k in d2: val= d1[k]-d2[k]
            elif k in d1: val= d1[k]
            elif k in d2: val= -d2[k]
            else: val = None
            return val
    
        return{k:getval(k) for k in d1|d2 if getval(k) != 0}
    
    def __add__(self, othersymb):
        if isinstance(othersymb,Symb): os=othersymb
        else: os=Symb(othersymb)
        s=self.dictmerge(self,os)
        #print(Symb(s))
        #print(s,os.mydict,self.mydict)
        return Symb(s)
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self, othersymb):
        return Symb(self.dictdiff(self,othersymb))

    def __rsub__(self, othersymb):
        return Symb(self.dictdiff(othersymb,self))

    def __mul__(self,const):
        return Symb(self.valmult(const,self))
    
    def __rmul__(self,const):
        return Symb(self.valmult(const,self))
    
    def __getitem__(self,key):
        if key in self: return super().__getitem__(key)
        else: return 0
    
class sumlist():
    def __init__(self,mylist):
        self.list=mylist
        
    def __add__(self, otherlist):
        if isinstance(otherlist,sumlist): os=otherlist.list
        else: os=otherlist
        
        outlist=[a_i + b_i for a_i, b_i in zip(self.list, os)]
        return sumlist(outlist)

    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self, otherlist):
        if isinstance(otherlist,sumlist): os=otherlist.list
        else: os=otherlist
        
        outlist=[a_i - b_i for a_i, b_i in zip(self.list, os)]
        return sumlist(outlist)

    def __rsub__(self, other):
        if isinstance(otherlist,sumlist): os=otherlist.list
        else: os=otherlist
        
        outlist=[a_i - b_i for a_i, b_i in zip(otherlist, os)]
        return sumlist(outlist)
    
    def __mul__(self,const):
        return [const*elem for elem in self.list]

    def __rmul__(self,const):
        return [const*elem for elem in self.list]
    
    def __getitem__(self,key):
        if key in self: return super().__getitem__(key)
        else: return 0

In [1]:
#preprocessing the data: read in the input FBspace files

def readFBspace(name='frontspace', w=2):
    assert os.path.isfile(f'../data/{name}')
    res = ''
    prefix = name

    with open(f'../data/{name}', 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix + '[' + str(w) + ']'): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res

def SB_to_dict(mystring):
    def to_coef(mystr):
        if mystr=='-': return -1
        elif mystr == '': return 1
        else: return int(mystr)
    m=mystring.replace('-','+-').replace(",","").split('+')
    m2=[el.replace("(","").replace(")","").replace("*","").split("SB") for el in m]
    sbdict={elem[1]:to_coef(elem[0]) for elem in m2 if len(elem) > 1}
    return sbdict

def convert(w, filename):
    mystr=''.join(str.split(readFBspace(name=filename, w=w)))
    newstr=re.split(":=|\[|\]",mystr)[4]
    #goodstr=re.split(",(SB)",newstr)
    #print(newstr)
    #print(re.sub('[,*]', '', readFBspace(name='frontspace', w=2)))
    dev = [elem+")" if elem[-1]!=")" else elem for elem in newstr.split("),") if elem]
    #print([el for el in dev])
    return [SB_to_dict(el) for el in dev]

def coefclip_to_ind(coefclip,wt,name):
    
    if "front" in name:
        num=F_number[wt]
    elif "back" in name:
        num=B_number[wt]
    else:
        print("Error! Bad name!")
        raise ValueError
        
    mystr=re.split(',|\[|\]', coefclip)[1:-1]
    if "front" in name:
        ind=(letters.index(mystr[1])*num)+(int(mystr[0])-1)
        if (int(mystr[0]) > num): return None
    else:
        ind=(letters.index(mystr[0])*num)+(int(mystr[1])-1)
        if (int(mystr[1]) > num): return None
    
    return ind

def ind_to_coefclip(ind,wt,name): 
    
    if "front" in name:
        num=F_number[wt]
    elif "back" in name:
        num=B_number[wt]
    else:
        print("Error! Bad name!")
        raise ValueError
    
    letter=letters[(ind // num)] 
    bspace=(ind % num)+1
    if "front" in name:
        mystr=f'c[{bspace},{letter}]'
    if "back" in name:
        mystr=f'c[{letter},{bspace}]'
    return mystr
    
def coefclip2_to_ind(coefclip,wt,name):
    
    if "front" in name:
        num=F_number[wt]
    elif "back" in name:
        num=B_number[wt]
    else:
        print("Error! Bad name!")
        raise ValueError
        
    mystr=re.split(',|\[|\]', coefclip)[1:-1]
    if "front" in name:
        ind=(letters.index(mystr[2])*6*num)+(letters.index(mystr[1])*num)+(int(mystr[0])-1)
        if (int(mystr[0]) > num): return None
    else:
        ind=(letters.index(mystr[0])*6*num)+(letters.index(mystr[1])*num)+(int(mystr[2])-1)
        if (int(mystr[2]) > num): return None
    
    return ind

def ind_to_coefclip2(ind,wt,name): 
    
    if "front" in name:
        num=F_number[wt]
    elif "back" in name:
        num=B_number[wt]
    else:
        print("Error! Bad name!")
        raise ValueError
        
    bspace=(ind % num)
    ind=(ind-bspace)//num
    letter2=ind%6
    ind=(ind-letter2)//6
    letter=ind

    if "front" in name:
        mystr=f'c[{bspace+1},{letters[letter2]},{letters[letter]}]'
    else:
        mystr=f'c[{letters[letter]},{letters[letter2]},{bspace+1}]'
        
    return mystr

In [ ]:
def frref(A, TOL=None):
    #custom fast pyTorch function to do Gauss-Jordan reduced row echelon form
    #reduce the set of equations to the unique minimal form
    m,n = A.shape
    if TOL is None:
        TOL = max(m, n)*torch.finfo(A.dtype).eps*torch.linalg.norm(A, np.inf)

    i = 0
    j = 0
    jb = []

    while (i < m) and (j < n):
        # Find value (p) and index (k) of largest element in the remainder of column j.
        abscol = torch.abs(A[i:m, j])
        p = torch.max(abscol)
        k = torch.argmax(abscol, axis=0)
        
        if k.ndim > 1:
            k = k[0]
        else:
            k = int(k)

        k = k+i

        
        if p <= TOL:
            # % The column is negligible, zero it out.
            zs= torch.zeros_like(A[i:m, j])
            A[i:m, j] = zs
            j += 1
        else:
            # Remember column index
            jb.append(j)
            
            # Swap i-th and k-th rows.
            myswap = A[[k, i], j:n]
            A[[i, k], j:n] = myswap
            
            # % Divide the pivot row by the pivot element.
            Ai = torch.nan_to_num(A[i, j:n] / A[i, j]).T.T

            # % Subtract multiples of the pivot row from all the other rows.
            elem=torch.matmul(A[:, [j]],Ai.unsqueeze(0))
            myval=A[:, j:n] - elem
            A[:, j:n] = myval
            
            A[i, j:n] = Ai
            i += 1
            j += 1

    return A, jb

In [ ]:
def sumstring(i,mystring, k, v): 
    if v == 1:
        if i ==0: mystring += f'{k}'
        else: mystring += f'+{k}'
    elif v == -1:
        mystring += f'-{k}'    
    elif v > 0:
        if i ==0: mystring += f'{v}*{k}'
        else: mystring += f'+{v}*{k}'
    elif v < 0: mystring += f'{v}*{k}'
    else: mystring=mystring
           
    return mystring

def rels_from_coeffs(rels,wt,name):
    if "front" in name:
        num=F_number[wt]
        reduce=False
    elif "back" in name:
        num=B_number[wt]
        reduce=True
   
    #get the list of dicts
    #encode the coefs as indices
    #write the eqs into a matrix
    M=[]
    for rel in rels:
        row=[0]*(6*num)
        for key in rel:
            row[coefclip_to_ind(key,wt,name)] = rel[key]
        M.append(row)
    
    M_gpu=(torch.from_numpy(np.array(M).astype('double')).to(device='cuda'))
    myrank=torch.linalg.matrix_rank(M_gpu).cpu().detach()
    
    out_rels=[]
    if reduce:
        #get the row echelon form of the matrix
        rmat=frref(M_gpu)
        mymat=rmat[0].cpu().detach().numpy().tolist()
    else:
        mymat=M
        
    #read off each row, get the coefs
    for row in mymat:
        rel=''
        counter=0
        for i,elem in enumerate(row):
            if elem == 0: continue
            k=ind_to_coefclip(i,wt,name)
            counter +=1
            the_elem=Fraction(elem).limit_denominator(1000000)
            rel = sumstring(counter,rel,k,the_elem)
            #print(rel)
        if rel != '':
            rel+='=0'
            out_rels.append(rel)
    #print(rels)
    return out_rels, myrank  
    
def rels_from_coeffs_2letter(rels,wt,name):
    if "front" in name:
        num=F_number[wt]
        reduce=False
    elif "back" in name:
        num=B_number[wt]
        reduce=True
        
    #get the list of dicts
    #encode the coefs as indices
    #write the eqs into a matrix
    M=[]
    for rel in rels:
        row=[0]*(36*num)
        for key in rel:
            row[coefclip2_to_ind(key,wt,name)] = rel[key]
        M.append(row)
    
    M_gpu=(torch.from_numpy(np.array(M).astype('double')).to(device='cuda'))
    myrank=torch.linalg.matrix_rank(M_gpu).cpu().detach()
    
    out_rels=[]
    if reduce:
        #get the row echelon form of the matrix
        rmat=frref(M_gpu)
        mymat=rmat[0].cpu().detach().numpy().tolist()
    else:
        mymat=M
        
    #read off each row, get the coefs
    for row in mymat:
        rel=''
        counter=0
        for i,elem in enumerate(row):
            if elem == 0: continue
            k=ind_to_coefclip2(i,wt,name)
            counter +=1
            the_elem=Fraction(elem).limit_denominator(1000000)
            rel = sumstring(counter,rel,k,the_elem)
            #print(rel)
        if rel != '':
            rel+='=0'
            out_rels.append(rel)
    #print(rels)
    return out_rels, myrank  

In [ ]:
letters=['a','b','c','d','e','f']
B_number= [0, 3, 6, 12, 24, 45, 85, 155, 289, None ] #<- dim_back
F_number= [0, 3, 9, 21, 48, 108, 246, 555, 1251, None ]

def B_all(wgt):
    return convert(wgt,"backspace")
    
def F_all(wgt):
    return convert(wgt,"frontspace")

#null for loop zero
fspaces = [{}]+[convert(i,"frontspace") for i in range(1,7)]
bspaces = [{}]+[convert(i,"backspace") for i in range(1,8)]
    
def MakeClipCoproducts(upper_wgt,name,do_zeros=False):
    #Given an Fspace or Bspace:
    #Clip off the letter and solve over the lower weight space.
    #This is the "three index tensor".
    #T[letter][upper_elem][lower_elem]: clip "letter" from upper_elem,
    #express in terms of linear combo of lower_elem.
    #Tensor gives coef of each lower_elem in this combo.
    
    if "front" in name:
        space=fspaces[upper_wgt]
        lowerspace=fspaces[upper_wgt-1]
    elif "back" in name:
        space=bspaces[upper_wgt]
        lowerspace=bspaces[upper_wgt-1]
    else:
        print("Error! Bad name!")
        raise ValueError
        
    allclips={}
    M=[]
    #make the coef matrix
    lowerkeys=[key for lower_elem in lowerspace for key in lower_elem.keys()]
    for key in lowerkeys:
        coef_list=[]
        for lower_elem in lowerspace:
            if key in lower_elem: coef_list.append(lower_elem[key])
            else: coef_list.append(0)
        M.append(coef_list)
    npM=torch.from_numpy(np.array(M).astype('double')).to(device='cuda')
        
    for letter in letters:
        higher_dict={}
        for j,elem in enumerate(space):
            clip_symb={}
            #For each element: clip off the letter
            for key in elem:
                if name=='backspace' and key[0] == letter:
                    clip_symb[key[1:]]=elem[key]
                elif name=='frontspace' and key[-1] == letter:
                    clip_symb[key[:-1]]=elem[key]

            #Do a lookup into the lower space and find all the clip_symb keys
            #Find the way to linearly-compose (dim_lower_space) elements to get the solution
            #y is just coef in higher-weight symb
            #This is a linear system
            y=[]

            for key in lowerkeys:
                if key in clip_symb: y.append(clip_symb[key])
                else: y.append(0)
            npy=torch.from_numpy(np.array(y).astype('double')).to(device='cuda')
            #print(npM,npy)
            outs=np.round(torch.linalg.lstsq(npM,npy)[0].cpu().numpy()).astype(int)
            if do_zeros:
                lower_dict={f'{upper_wgt-1}_{i+1}':outs[i] for i in range(len(lowerspace))}
            else:
                lower_dict={f'{upper_wgt-1}_{i+1}':outs[i] for i in range(len(lowerspace)) if outs[i] != 0}
            if (not (set(lower_dict.values()) == {0} or len(set(lower_dict.values()))==0) and not do_zeros):
                higher_dict[f'{upper_wgt}_{j+1}']=lower_dict
            else:
                if do_zeros: 
                    higher_dict[f'{upper_wgt}_{j+1}']=lower_dict
        allclips[letter]=higher_dict    
    return allclips
    
def BCop(letter,upper_wgt,name,do_zeros=False):
    #get all lower wgts in terms of higher wgts (Lance's 's[i]')
    #this is equivalent to swapping the non-letter indices of the 3-index tensor
    if "front" in name: sp_number= F_number
    else:sp_number= B_number
    Bcop_dict={}
    MCC=MakeClipCoproducts(upper_wgt,name,do_zeros=do_zeros)[letter]
    #print(MCC)
    for lower in [f'{upper_wgt-1}_{i+1}' for i in range(sp_number[upper_wgt-1])]:
        Bcop_dict[lower]={}
        for upper in [f'{upper_wgt}_{j+1}' for j in range(sp_number[upper_wgt])]:
            if upper in MCC and lower in MCC[upper]:
                Bcop_dict[lower][upper]=MCC[upper][lower]
    return Bcop_dict

#Bcop: takes args (letter, expr, wgt)
#gets coefs of each elem in expr (c[elem_up]) and plugs in for s[k in upper loop].
#returns: (c[letter,elem_up]+....+)B(lower,elem1)+...+c[letter,elem_low]B(lower,elemN) etc.)


#be careful here: if clipping off Fspace, clip last letter, then next to last
#but if bspace, clip first, then last

def BCop2ABIG(let1,let2,upper_wt,name):  
    #inputs: sum(c[letter,j]B(w-1,j)) for j in back[w-1]
    #Bcop(): gets coef of BS elem 
    #per Lance notation, let2 is the one in the seam
    
    bc2=BCop(let1,upper_wt-1,name)
    #print(bc2)
    if "front" in name: sp_number= F_number
    else:sp_number= B_number
    
    out_eq={}
    for bs,coefs in bc2.items():
        out_eq[bs]=Symb()   
        for j in range(1,sp_number[upper_wt-1]+1):
            if f'{upper_wt-1}_{j}' in bc2[bs]:
                if "front" in name: mystr=f'c[{j},{let2}]'
                else:mystr=f'c[{let2},{j}]'
                out_eq[bs][mystr] = bc2[bs][f'{upper_wt-1}_{j}']
    s=sumlist([Symb(v) for v in out_eq.values()])
    return s

def BCop2(firstclip,secondclip,upper_wgt,name,do_zeros=False):
    #clip the first letter,l1. This gives an expression 
    #of the form (s[0]+s[1]+s[2])B(3,1)+(s[0]+s[1]+s[2])B(3,2), etc.

    #Then clip the second letter. This gives an expression of the form
    #(s[0]+s[1]+s[2])B(2,1)+(s[0]+s[1]+s[2])B(2,2)

    #Take ex. coef B(3,1) from the first expression. Sub for s[1] in the second expression.

    bcop1=BCop(firstclip,upper_wgt,name,do_zeros=do_zeros)
    bcop2=BCop(secondclip,upper_wgt-1,name,do_zeros=do_zeros)
    #print(bcop2)
    dcops={}
    for cop2, val2 in bcop2.items():
            dcop=Symb({})
            for v,vcoef in val2.items():
                dcop+= vcoef*Symb(bcop1[v])
                
                #thisdict={myk:vcoef*myv for myk,myv in bcop1[v].items()}
                #dcop=dictmerge(thisdict,dcop)
            dcops[cop2]=dcop
    return dcops


def BCop3ABIG(let1,let2,let3,upper_wt,name):
    
    #inputs: sum(c[letter,j]B(w-1,j)) for j in back[w-1]
    #Bcop(): gets coef of BS elem 
    #per Lance notation, let3 is the one in the seam
    if "front" in name: sp_number= F_number
    else:sp_number= B_number
    
    bc2=BCop2(let1,let2,upper_wt-1,name)
    out_eq={}
    for bs,coefs in bc2.items():
        out_eq[bs]=Symb()   
        for j in range(1,sp_number[upper_wt-1]+1):
            if f'{upper_wt-1}_{j}' in bc2[bs]:
                if "front" in name: mystr=f'c[{j},{let3}]'
                else:mystr=f'c[{let3},{j}]'
                out_eq[bs][mystr] = bc2[bs][f'{upper_wt-1}_{j}']
    s=sumlist([Symb(v) for v in out_eq.values()])
    return s

def BCop3ABIG_2span(let1,let2,let3,upper_wt,name):
    
    #inputs: sum(c[letter,j]B(w-2,j)) for j in back[w-2]
    #Bcop(): gets coef of BS elem 
    #per Lance notation, let3 is the one in the seam
    
    
    #Vanilla: clip let1 & let2, get coefs. Here, just clip let1
    
    if "front" in name: sp_number= F_number
    else:sp_number= B_number
    
    bc2=BCop(let1,upper_wt-1,name)
    out_eq={}
    for bs,coefs in bc2.items():
        out_eq[bs]=Symb()   
        for j in range(1,sp_number[upper_wt-1]+1):
            if f'{upper_wt-1}_{j}' in bc2[bs]:
                if "front" in name: mystr=f'c[{j},{let2},{let3}]'
                else:mystr=f'c[{let3},{let2},{j}]'
                out_eq[bs][mystr] = bc2[bs][f'{upper_wt-1}_{j}']
    s=sumlist([Symb(v) for v in out_eq.values()])
    return s

In [ ]:
def testpairABIG(wt,name):
    #shift up a loop to match Lance notation
    #wt=wt+1
    rels_list=[]
    rels_list += (BCop2ABIG("a","d",wt,name)).list
    rels_list += (BCop2ABIG("d","a",wt,name)).list
    rels_list += (BCop2ABIG("b","e",wt,name)).list
    rels_list += (BCop2ABIG("e","b",wt,name)).list
    rels_list += (BCop2ABIG("c","f",wt,name)).list
    rels_list += (BCop2ABIG("f","c",wt,name)).list
    rels_list += (BCop2ABIG("e","d",wt,name)).list
    rels_list += (BCop2ABIG("d","e",wt,name)).list
    rels_list += (BCop2ABIG("f","d",wt,name)).list
    rels_list += (BCop2ABIG("d","f",wt,name)).list
    rels_list += (BCop2ABIG("f","e",wt,name)).list
    rels_list += (BCop2ABIG("e","f",wt,name)).list
    
    return rels_list

def integs(wt,name):
    rels_list=[]
    print('now working on pair 1 of 3',wt,name)
    rels_list += (BCop2ABIG('a','b',wt,name)+BCop2ABIG('a','c',wt,name)-BCop2ABIG('b','a',wt,name)
                                  -BCop2ABIG('c','a',wt,name)).list
    print('now working on pair 2 of 3',wt,name)
    rels_list += (BCop2ABIG('c','a',wt,name)+BCop2ABIG('c','b',wt,name)-BCop2ABIG('a','c',wt,name)
                                  -BCop2ABIG('b','c',wt,name)).list
    
    print('now working on pair 3 of 3',wt,name)
    rels_list += (BCop2ABIG('d','b',wt,name)-BCop2ABIG('d','c',wt,name)-BCop2ABIG('b','d',wt,name)
                                  +BCop2ABIG('c','d',wt,name)+BCop2ABIG('e','c',wt,name)-BCop2ABIG('e','a',wt,name)
                                  -BCop2ABIG('c','e',wt,name)+BCop2ABIG('a','e',wt,name)+BCop2ABIG('f','a',wt,name)
                                  -BCop2ABIG('f','b',wt,name)-BCop2ABIG('a','f',wt,name)+BCop2ABIG('b','f',wt,name)
                                  +2*(BCop2ABIG('c','b',wt,name)-BCop2ABIG('b','c',wt,name))).list

    return rels_list

def triples(wt,name):
    rels_list=[]
    print('now working on triple 1 of 6',wt,name)
    rels_list += (BCop3ABIG('a','a','b',wt,name)+BCop3ABIG('a','b','b',wt,name)
                                  +BCop3ABIG('a','c','b',wt,name)).list                       
    print('now working on triple 2 of 6',wt,name)
    rels_list += (BCop3ABIG('a','a','c',wt,name)+BCop3ABIG('a','b','c',wt,name)
                                  +BCop3ABIG('a','c','c',wt,name)).list
                                  
    print('now working on triple 3 of 6',wt,name)
    rels_list += (BCop3ABIG('b','a','a',wt,name)+BCop3ABIG('b','b','a',wt,name)
                                  +BCop3ABIG('b','c','a',wt,name)).list   
                                  
    print('now working on triple 4 of 6',wt,name)
    rels_list += (BCop3ABIG('b','a','c',wt,name)+BCop3ABIG('b','b','c',wt,name)
                                  +BCop3ABIG('b','c','c',wt,name)).list       
                                  
    print('now working on triple 5 of 6',wt,name)
    rels_list += (BCop3ABIG('c','a','b',wt,name)+BCop3ABIG('c','b','b',wt,name)
                                  +BCop3ABIG('c','c','b',wt,name)).list             
                                  
    print('now working on triple 6 of 6',wt,name)
    rels_list += (BCop3ABIG('c','a','a',wt,name)+BCop3ABIG('c','b','a',wt,name)
                                  +BCop3ABIG('c','c','a',wt,name)).list  
    return rels_list


def triples2span(wt,name):
    #check this!
    rels_list=[]
    print('now working on triple 1 of 6',wt,name)
    rels_list += (BCop3ABIG_2span('a','a','b',wt,name)+BCop3ABIG_2span('a','b','b',wt,name)
                                  +BCop3ABIG_2span('a','c','b',wt,name)).list                       
    print('now working on triple 2 of 6',wt,name)
    rels_list += (BCop3ABIG_2span('a','a','c',wt,name)+BCop3ABIG_2span('a','b','c',wt,name)
                                  +BCop3ABIG_2span('a','c','c',wt,name)).list
                                  
    print('now working on triple 3 of 6',wt,name)
    rels_list += (BCop3ABIG_2span('b','a','a',wt,name)+BCop3ABIG_2span('b','b','a',wt,name)
                                  +BCop3ABIG_2span('b','c','a',wt,name)).list    
                                  
    print('now working on triple 4 of 6',wt,name)
    rels_list += (BCop3ABIG_2span('b','a','c',wt,name)+BCop3ABIG_2span('b','b','c',wt,name)
                                  +BCop3ABIG_2span('b','c','c',wt,name)).list       
                                  
    print('now working on triple 5 of 6',wt,name)
    rels_list += (BCop3ABIG_2span('c','a','b',wt,name)+BCop3ABIG_2span('c','b','b',wt,name)
                                  +BCop3ABIG_2span('c','c','b',wt,name)).list             
                                  
    print('now working on triple 6 of 6',wt,name)
    rels_list += (BCop3ABIG_2span('c','a','a',wt,name)+BCop3ABIG_2span('c','b','a',wt,name)
                                  +BCop3ABIG_2span('c','c','a',wt,name)).list  
    return rels_list

def all_rels_onespan(wt,name):
    myrels=testpairABIG(wt,name)+integs(wt,name)
    if wt > 3:
        myrels=myrels+triples(wt,name)
    rels,relrank=rels_from_coeffs(myrels,wt,name)
    return rels,relrank

def all_rels_2span(wt,name):
    myrels2=triples2span(wt,name)
    rels2,relrank=rels_from_coeffs_2letter(myrels2,wt,name)
    return rels2,relrank

In [ ]:
#backspace, vanilla
for i in [3,4,5,6,7,8]:
    r,rk=all_rels_onespan(i,"backspace")
    print(f'rels_backspace_{i-1}: {len(r)} single-coproduct rels, rank {rk}')
    print(r)

In [ ]:
#frontspace, 2_exposed

for i in [3,4,5,6,7,8]:
    r,rk=all_rels_2span(i,"backspace")
    print(f'rels_backspace_{i-1}: {len(r)} double-coproduct rels, rank {rk}')
    print(r)

In [ ]:
#frontspace, vanilla
for i in [3,4,5,6,7]:
    r,rk=all_rels_onespan(i,"frontspace")
    print(f'rels_frontspace_{i-1}: {len(r)} single-coproduct rels, rank {rk}')
    print(r)

In [ ]:
#frontspace, 2_exposed

for i in [3,4,5,6,7]:
    r,rk=all_rels_2span(i,"frontspace")
    print(f'rels_frontspace_{i-1}: {len(r)} double-coproduct rels, rank {rk}')
    print(r)

In [ ]:
#do these by hand, coproduct function fails here.
#works ok in maple, so may be worth revisiting
print(B_all(1),F_all(1))

so:

Bspace_rels(1)=['+c[a,1]=0', '+c[b,2]=0', '+c[c,3]=0', '+c[e,1]=0', '+c[e,3]=0', '+c[d,2]=0', '+c[d,3]=0', '+c[f,1]=0', '+c[f,2]=0']

and

Fspace_rels(1)=['+c[1,d]'=0, '+c[2,e]=0', +'c[3,f]=0', '+c[1,b]+c[1,c]-c[2,a]-c[3,a]=0', '+c[3,a]+c[3,b]-c[1,c]-c[2,c]=0', '+c[2,f]−c[2,d]+c[3,d]−c[3,e]+c[1,e]−c[1,f]+4*c[3,b]−4*c[2,c]=0']


